In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dropout, Dense
from keras.layers import Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
file_dir = "C:/data/"
image_size = (224, 224)
cartegories  = os.listdir(file_dir)
num_classes = len(cartegories)
X=[]
Y=[]

print(cartegories)
# 이미지 전처리
for i, categorie in  enumerate(cartegories):
    label = [0 for i in range(num_classes)]
    label[i] = 1
    print(categorie)
    img_dir =  file_dir+categorie +"/"
    img_list = os.listdir(img_dir)
    for img_list_dir in img_list:
        img_path = img_dir+img_list_dir
        img = cv2.imdecode(np.fromfile(img_path, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img, image_size)
        X.append(img / 255)
        Y.append(label)
        print(img_path)
X=np.array(X)
Y=np.array(Y)

print(X.shape)
print(X.shape[1:])

In [ ]:
train_input, test_input, train_target, test_target = train_test_split(X,Y)

print( train_input.shape, train_target.shape)
print( test_input.shape, test_target.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                 input_shape=train_input.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Classifier
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(9, activation='softmax'))


model.summary()

In [ ]:
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
modelpath= MODEL_DIR +".epoch{epoch:02d}-loss{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1,
save_best_only=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_input, train_target, validation_data=(test_input, test_target), epochs=200, batch_size=16,
verbose=1, callbacks=[early_stopping_callback,checkpointer])
print("\n Test Accuracy: %.4f" % (model.evaluate(test_input, test_target)[1])) 


In [ ]:
#그래프 그려보기
y_vloss=history.history['val_loss']
y_loss =history.history["loss"]
y_acc  =history.history["accuracy"]

x_len= np.arange(len(y_loss))
plt.plot(x_len,y_vloss,marker='.', c="r",label="Testset_loss")
plt.plot(x_len, y_loss,marker='.', c="b", label="Tarinset_loss")
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()